# Multinational Retail Data Centralisation Project

## Aim

<b>To go through what its like to source, clean and centralise retail sales data to a database which can then be queried and for analysis.</b>

## In this project I will learn to:

1. Extract data from various sources like AWS RDS, AWS S3 and PDF file
2. Clean each extracted data
3. Push the cleaned data as tables to a database
4. Create the star-schema model in your database
5. Query the database for sales analysis

## Key Prerequisites

- <b>OOP:</b> Classes, methods and functions.
- <b>Pandas:</b> Reading from data sources, cleaning data and pushing data to a database
- <b>AWS:</b> boto3 for code and CLI for configuration
- <b>SQL:</b> For creating data model and querying the database

## Other prerequisites

- <b>APIs:</b> For extracting the stores data
- <b>tabula</b> (and installation + configuration of Java): For extracting the credit card data in PDF file
- <b>Data and file types:</b> YAML, JSON, CSV

## Software needed

- <b>VSCode:</b> IDE (Integrated Development Environment)
- <b>Conda:</b> Package manager
- <b>pgAdmin4 or SQLTools:</b> Interface for PostgreSQL

## TASK 0

### Import Required Libraries

In [1]:
from class_1_data_extractor import DataExtractor
from class_2_database_connector import DatabaseConnector
from class_3_data_cleaning import DataCleaning
import pandas as pd
import re
import requests

## TASK 1


### Connect to the RDS Database and Extract Data

In [2]:
# Step 1: Connect to the RDS Database
rds_db_connector = DatabaseConnector(config_path='aws_db_creds.yaml')

# Step 2: Extract Data from RDS
data_extractor = DataExtractor(rds_db_connector)
tables = data_extractor.list_tables()

# Specify the table you want to clean
target_table = 'legacy_users'

if target_table in tables:
    df = data_extractor.read_rds_table(target_table)
    if df is not None:
        print("Data before cleaning:")
        display(df.head())  # Use display to render the DataFrame nicely in Jupyter
        # Save the DataFrame as 'legacy_dim_users'
        df.to_csv('legacy_dim_users.csv', index=False)
        print("DataFrame saved as 'legacy_dim_users.csv'")
    else:
        print("Failed to extract the table data")
else:
    print(f"Table {target_table} not found in the database.")

Database engine initialized successfully.
Tables in the database: ['legacy_store_details', 'dim_card_details', 'legacy_users', 'orders_table']
Data before cleaning:


,index,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
0,0,Sigfried,Noack,1990-09-30,Heydrich Junitz KG,rudi79@winkler.de,Zimmerstr. 1/0\n59015 Gießen,Germany,DE,+49(0) 047905356,2018-10-10,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8
1,1,Guy,Allen,1940-12-01,Fox Ltd,rhodesclifford@henderson.com,Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH,United Kingdom,GB,(0161) 496 0674,2001-12-20,8fe96c3a-d62d-4eb5-b313-cf12d9126a49
2,2,Harry,Lawrence,1995-08-02,"Johnson, Jones and Harris",glen98@bryant-marshall.co.uk,92 Ann drive\nJoanborough\nSK0 6LR,United Kingdom,GB,+44(0)121 4960340,2016-12-16,fc461df4-b919-48b2-909e-55c95a03fe6b
3,3,Darren,Hussain,1972-09-23,Wheeler LLC,daniellebryan@thompson.org,19 Robinson meadow\nNew Tracy\nW22 2QG,United Kingdom,GB,(0306) 999 0871,2004-02-23,6104719f-ef14-4b09-bf04-fb0c4620acb0
4,4,Garry,Stone,1952-12-20,Warner Inc,billy14@long-warren.com,3 White pass\nHunterborough\nNN96 4UE,United Kingdom,GB,0121 496 0225,2006-09-01,9523a6d3-b2dd-4670-a51a-36aebc89f579


DataFrame saved as 'legacy_dim_users.csv'


### Clean the Data

In [3]:
if 'df' in locals():
    # Step 3: Clean the Data
    data_cleaner = DataCleaning()
    cleaned_df = data_cleaner.clean_user_data(df)
    cleaned_df = cleaned_df.applymap(lambda x: re.sub(r',\s*', ', ', x) if isinstance(x, str) else x)
    print("Data after cleaning:")
    display(cleaned_df.head())  # Use display to render the DataFrame nicely in Jupyter

Data after cleaning:


/var/folders/c5/5hv48d2x71q5swk5cwj45zy40000gn/T/ipykernel_39827/4085258179.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cleaned_df = cleaned_df.applymap(lambda x: re.sub(r',\s*', ', ', x) if isinstance(x, str) else x)


,index,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
0,0,Sigfried,Noack,1990-09-30,Heydrich Junitz KG,rudi79@winkler.de,"Zimmerstr. 1/0, 59015 Gießen",Germany,DE,490047905356,2018-10-10,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8
1,1,Guy,Allen,1940-12-01,Fox Ltd,rhodesclifford@henderson.com,"Studio 22a, Lynne terrace, McCarthymouth, TF0 9GH",United Kingdom,GB,01614960674,2001-12-20,8fe96c3a-d62d-4eb5-b313-cf12d9126a49
2,2,Harry,Lawrence,1995-08-02,"Johnson, Jones and Harris",glen98@bryant-marshall.co.uk,"92 Ann drive, Joanborough, SK0 6LR",United Kingdom,GB,4401214960340,2016-12-16,fc461df4-b919-48b2-909e-55c95a03fe6b
3,3,Darren,Hussain,1972-09-23,Wheeler LLC,daniellebryan@thompson.org,"19 Robinson meadow, New Tracy, W22 2QG",United Kingdom,GB,03069990871,2004-02-23,6104719f-ef14-4b09-bf04-fb0c4620acb0
4,4,Garry,Stone,1952-12-20,Warner Inc,billy14@long-warren.com,"3 White pass, Hunterborough, NN96 4UE",United Kingdom,GB,01214960225,2006-09-01,9523a6d3-b2dd-4670-a51a-36aebc89f579


### Connect to Local Database and Upload the Cleaned Data

In [4]:
if 'cleaned_df' in locals():
    # Step 4: Connect to the Local Database
    local_db_connector = DatabaseConnector(config_path='local_db_creds.yaml')

    # Step 5: Upload the cleaned data to the local database as 'dim_users'
    local_db_connector.upload_to_db(cleaned_df, "dim_users")
    print("Data uploaded to the local database as 'dim_users'")

    # Upload the legacy_dim_users dataframe
    legend_dim_users = pd.read_csv('legacy_dim_users.csv')
    local_db_connector.upload_to_db(legend_dim_users, "legacy_dim_users")
    print("Legend data uploaded to the local database as 'legacy_dim_users'")
else:
    print("Cleaned DataFrame not found, cannot upload to the database.")

Database engine initialized successfully.
Data uploaded to table dim_users successfully.
Data uploaded to the local database as 'dim_users'
Data uploaded to table legacy_dim_users successfully.
Legend data uploaded to the local database as 'legacy_dim_users'


## TASK 2


### Retrieve Data from the PDF

In [5]:
# Step 1: Retrieve data from the PDF
pdf_link = "https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf"
data_extractor = DataExtractor()

# Extract data from the PDF
pdf_data_df = data_extractor.retrieve_pdf_data(pdf_link)

if not pdf_data_df.empty:
    print("Data after extraction and attempted conversion:")
    display(pdf_data_df.head())  # Assuming you are in Jupyter, this will nicely render the DataFrame
else:
    print("Failed to retrieve data from the PDF.")

# Save the DataFrame as 'legacy_dim_card_details'
pdf_data_df.to_csv('legacy_dim_card_details.csv', index=False)
print("DataFrame saved as 'legacy_dim_card_details.csv'")

Failed to import jpype dependencies. Fallback to subprocess.
No module named 'jpype'


Data after extraction and attempted conversion:


,card_number,expiry_date,card_provider,date_payment_confirmed
0,30060773296197,09/26,Diners Club / Carte Blanche,2015-11-25
1,349624180933183,10/23,American Express,2001-06-18
2,3529023891650490,06/23,JCB 16 digit,2000-12-26
3,213142929492281,09/27,JCB 15 digit,2011-02-12
4,502067329974,10/25,Maestro,1997-03-13


DataFrame saved as 'legacy_dim_card_details.csv'


### Standardise Null Values

In [6]:
# Step 2.1: Standardize Nulls
if 'pdf_data_df' in locals():
    data_cleaner = DataCleaning()
    cleaned_df = data_cleaner.standardize_nulls(pdf_data_df)
    print("Data after standardizing nulls:")
    display(cleaned_df.head())
else:
    print("Data extraction failed, cannot proceed with cleaning.")

Data after standardizing nulls:


,card_number,expiry_date,card_provider,date_payment_confirmed
0,30060773296197,09/26,Diners Club / Carte Blanche,2015-11-25
1,349624180933183,10/23,American Express,2001-06-18
2,3529023891650490,06/23,JCB 16 digit,2000-12-26
3,213142929492281,09/27,JCB 15 digit,2011-02-12
4,502067329974,10/25,Maestro,1997-03-13


### Clean Card Numbers

In [7]:
# Step 2.2: Clean Card Numbers
if 'cleaned_df' in locals():
    cleaned_df = data_cleaner.clean_card_number(cleaned_df)
    print("Data after cleaning card numbers:")
    display(cleaned_df.head())
else:
    print("Standardized null data not available, cannot proceed.")

Data after cleaning card numbers:


,card_number,expiry_date,card_provider,date_payment_confirmed
0,30060773296197,09/26,Diners Club / Carte Blanche,2015-11-25
1,349624180933183,10/23,American Express,2001-06-18
2,3529023891650490,06/23,JCB 16 digit,2000-12-26
3,213142929492281,09/27,JCB 15 digit,2011-02-12
4,502067329974,10/25,Maestro,1997-03-13


### Clean Dates

In [8]:
# Step 2.3: Clean Dates
if 'cleaned_df' in locals():
    cleaned_df = data_cleaner.clean_dates(cleaned_df, date_columns=['date_payment_confirmed'])
    print("Data after cleaning dates:")
    display(cleaned_df.head())
else:
    print("Card numbers not cleaned, cannot proceed.")

Data after cleaning dates:


,card_number,expiry_date,card_provider,date_payment_confirmed
0,30060773296197,09/26,Diners Club / Carte Blanche,2015-11-25
1,349624180933183,10/23,American Express,2001-06-18
2,3529023891650490,06/23,JCB 16 digit,2000-12-26
3,213142929492281,09/27,JCB 15 digit,2011-02-12
4,502067329974,10/25,Maestro,1997-03-13


### Remove Invalid Rows

In [9]:
# Step 2.4: Remove Invalid Rows
if 'cleaned_df' in locals():
    cleaned_df = data_cleaner.remove_invalid_rows(cleaned_df)
    print("Data after removing invalid rows:")
    display(cleaned_df.head())
else:
    print("Date cleaning not performed, cannot proceed.")

Data after removing invalid rows:


,card_number,expiry_date,card_provider,date_payment_confirmed
0,30060773296197,09/26,Diners Club / Carte Blanche,2015-11-25
1,349624180933183,10/23,American Express,2001-06-18
2,3529023891650490,06/23,JCB 16 digit,2000-12-26
3,213142929492281,09/27,JCB 15 digit,2011-02-12
4,502067329974,10/25,Maestro,1997-03-13


### Final Cleaned Data

In [10]:
# Display final cleaned data
if 'cleaned_df' in locals():
    print("Final cleaned data:")
    display(cleaned_df.head())
else:
    print("Invalid rows not removed, cannot proceed.")

Final cleaned data:


,card_number,expiry_date,card_provider,date_payment_confirmed
0,30060773296197,09/26,Diners Club / Carte Blanche,2015-11-25
1,349624180933183,10/23,American Express,2001-06-18
2,3529023891650490,06/23,JCB 16 digit,2000-12-26
3,213142929492281,09/27,JCB 15 digit,2011-02-12
4,502067329974,10/25,Maestro,1997-03-13


### Connect to Local Database and Upload Cleaned Data

In [11]:
# Step 3: Connect to Local Database
if 'cleaned_df' in locals():
    local_db_connector = DatabaseConnector(config_path='local_db_creds.yaml')

    # Step 4: Upload cleaned data to local database
    local_db_connector.upload_to_db(cleaned_df, "dim_card_details")
    print("Cleaned data uploaded to the local database as 'dim_card_details'")

    # Upload the legacy_dim_card_details dataframe
    legend_dim_card_details = pd.read_csv('legacy_dim_card_details.csv')
    local_db_connector.upload_to_db(legend_dim_card_details, "legacy_dim_card_details")
    print("Legend data uploaded to the local database as 'legacy_dim_card_details'")
else:
    print("Cleaned data not available, cannot upload.")

Database engine initialized successfully.
Data uploaded to table dim_card_details successfully.
Cleaned data uploaded to the local database as 'dim_card_details'
Data uploaded to table legacy_dim_card_details successfully.
Legend data uploaded to the local database as 'legacy_dim_card_details'


## TASK 3


### Define API Details and Headers

In [12]:
# Step 1: Define API details
stores_endpoint = "https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores"
store_details_endpoint = "https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details"
headers = {"x-api-key": "yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX"}

### Create an Instance of DataExtractor and Retrieve Number of Stores

In [13]:
# Step 2: Create an instance of DataExtractor
data_extractor = DataExtractor()

# Step 3: Retrieve the number of stores
number_of_stores = data_extractor.list_number_of_stores(stores_endpoint, headers)
print(f"Number of stores: {number_of_stores}")

Number of stores: 451


### Retrieve Stores Data

In [14]:
# Step 4: Retrieve stores data
if number_of_stores:
    stores_df = data_extractor.retrieve_stores_data(store_details_endpoint, headers, number_of_stores)
    if stores_df is not None:
        print("Data before cleaning:")
        display(stores_df.head())  # Use display to render the DataFrame in Jupyter
        
        # Save the DataFrame as 'legacy_dim_store_details'
        stores_df.to_csv('legacy_dim_store_details.csv', index=False)
        print("DataFrame saved as 'legacy_dim_store_details.csv'")
    else:
        print("Failed to retrieve stores data.")
else:
    print("Failed to retrieve number of stores.")

Error retrieving data for store number 451: 500 Server Error: Internal Server Error for url: https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/451
Data before cleaning:


,index,address,longitude,lat,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
0,1,"Flat 72W\nSally isle\nEast Deantown\nE7B 8EB, ...",51.62907,None,High Wycombe,HI-9B97EE4E,34,1996-10-25,Local,-0.74934,GB,Europe
1,2,"Heckerstraße 4/5\n50491 Säckingen, Landshut",48.52961,None,Landshut,LA-0772C7B9,92,2013-04-12,Super Store,12.16179,DE,Europe
2,3,"5 Harrison tunnel\nSouth Lydia\nWC9 2BE, Westbury",51.26,None,Westbury,WE-1DE82CEE,69,2014-01-02,Super Store,-2.1875,GB,Europe
3,4,Studio 6\nStephen landing\nSouth Simon\nB77 2W...,53.0233,None,Belper,BE-18074576,35,2019-09-09,Local,-1.48119,GB,Europe
4,5,Flat 92u\nChristian harbors\nPort Charlotte\nN...,53.38333,None,Gainsborough,GA-CAD01AC2,36,1995-05-15,Local,-0.76667,GB,Europe


DataFrame saved as 'legacy_dim_store_details.csv'


### Clean the Data

In [15]:
# Step 5: Clean the Data
if 'stores_df' in locals():
    data_cleaner = DataCleaning()
    cleaned_df = data_cleaner.clean_store_details(stores_df)
    cleaned_df = cleaned_df.applymap(lambda x: re.sub(r',\s*', ', ', x) if isinstance(x, str) else x)

    # Step 6: Display the cleaned data
    print("Data after cleaning:")
    display(cleaned_df.head())
else:
    print("Stores DataFrame not available, cannot proceed with cleaning.")

# Print the total number of rows
print(f"Total number of rows: {cleaned_df.shape[0]} out of 441.")

Data after cleaning:


/var/folders/c5/5hv48d2x71q5swk5cwj45zy40000gn/T/ipykernel_39827/3638795891.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cleaned_df = cleaned_df.applymap(lambda x: re.sub(r',\s*', ', ', x) if isinstance(x, str) else x)


,index,address,longitude,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
0,1,"Flat 72W, Sally isle, East Deantown, E7B 8EB, ...",51.62907,High Wycombe,HI-9B97EE4E,34,1996-10-25,Local,-0.74934,GB,Europe
1,2,"Heckerstraße 4/5, 50491 Säckingen, Landshut",48.52961,Landshut,LA-0772C7B9,92,2013-04-12,Super Store,12.16179,DE,Europe
2,3,"5 Harrison tunnel, South Lydia, WC9 2BE, Westbury",51.26,Westbury,WE-1DE82CEE,69,2014-01-02,Super Store,-2.1875,GB,Europe
3,4,"Studio 6, Stephen landing, South Simon, B77 2W...",53.0233,Belper,BE-18074576,35,2019-09-09,Local,-1.48119,GB,Europe
4,5,"Flat 92u, Christian harbors, Port Charlotte, N...",53.38333,Gainsborough,GA-CAD01AC2,36,1995-05-15,Local,-0.76667,GB,Europe


Total number of rows: 447 out of 441.


### Connect to Local Database and Upload Cleaned Data

In [16]:
# Step 7: Connect to the Local Database
if 'cleaned_df' in locals():
    local_db_connector = DatabaseConnector(config_path='local_db_creds.yaml')

    # Step 8: Upload the cleaned data to the local database
    local_db_connector.upload_to_db(cleaned_df, "dim_store_details")
    print("Cleaned data uploaded to the local database as 'dim_store_details'")

    # Upload the legacy_dim_store_details dataframe
    legend_dim_store_details = pd.read_csv('legacy_dim_store_details.csv')
    local_db_connector.upload_to_db(legend_dim_store_details, "legacy_dim_store_details")
    print("Legend data uploaded to the local database as 'legacy_dim_store_details'")
else:
    print("Cleaned data not available, cannot upload.")

Database engine initialized successfully.
Data uploaded to table dim_store_details successfully.
Cleaned data uploaded to the local database as 'dim_store_details'
Data uploaded to table legacy_dim_store_details successfully.
Legend data uploaded to the local database as 'legacy_dim_store_details'


## TASK 4


### Define S3 URI and Create DataExtractor Instance

In [17]:
# Step 1: Define S3 URI
s3_uri = 's3://data-handling-public/products.csv'

# Step 2: Create an instance of DataExtractor
data_extractor = DataExtractor()

### Retrieve Product Data from S3 bucket

In [18]:
# Step 3: Retrieve product data from S3
products_df = data_extractor.extract_from_s3(s3_uri)

# Check if the DataFrame was retrieved successfully
if products_df is not None:
    print("Data before cleaning:")
    display(products_df.head())  # Use display to render the DataFrame in Jupyter
    # Save the DataFrame as 'legend_dim_products'
    products_df.to_csv('legacy_dim_products.csv', index=False)
    print("DataFrame saved as 'legacy_dim_products.csv'")
else:
    print("Failed to retrieve product data from S3.")

Data before cleaning:


,Unnamed: 0,product_name,product_price,weight,category,EAN,date_added,uuid,removed,product_code
0,0,FurReal Dazzlin' Dimples My Playful Dolphin,£39.99,1.6kg,toys-and-games,7425710935115,2005-12-02,83dc0a69-f96f-4c34-bcb7-928acae19a94,Still_avaliable,R7-3126933h
1,1,Tiffany's World Day Out At The Park,£12.99,0.48kg,toys-and-games,487128731892,2006-01-09,712254d7-aea7-4310-aff8-8bcdd0aec7ff,Still_avaliable,C2-7287916l
2,2,Tiffany's World Pups Picnic Playset,£7.00,590g,toys-and-games,1945816904649,1997-03-29,b089ef6f-b628-4e37-811d-fffe0102ba64,Still_avaliable,S7-1175877v
3,3,Tiffany's World Wildlife Park Adventures,£12.99,540g,toys-and-games,1569790890899,2013-03-20,d55de422-8b98-47d6-9991-e4bc4c5c0cb0,Removed,D8-8421505n
4,4,Cosatto Cosy Dolls Pram,£30.00,1.91kg,toys-and-games,7142740213920,2007-12-23,7945b657-cb02-4cc5-96cf-f65ed0a8f235,Still_avaliable,B6-2596063a


DataFrame saved as 'legacy_dim_products.csv'


### Clean the Product Data

In [19]:
# Step 4: Clean the Data
if 'products_df' in locals():
    data_cleaner = DataCleaning()
    cleaned_df = data_cleaner.clean_product_data(products_df)

    # Step 5: Display the cleaned data
    print("Data after cleaning:")
    display(cleaned_df.head())
else:
    print("Product data not available, cannot proceed with cleaning.")

Data after cleaning:


,Unnamed: 0,product_name,product_price_gbp,category,EAN,date_added,uuid,removed,product_code,weight_kg
0,0,FurReal Dazzlin' Dimples My Playful Dolphin,39.99,toys-and-games,7425710935115,2005-12-02,83dc0a69-f96f-4c34-bcb7-928acae19a94,Still_avaliable,R7-3126933h,1.60
1,1,Tiffany's World Day Out At The Park,12.99,toys-and-games,487128731892,2006-01-09,712254d7-aea7-4310-aff8-8bcdd0aec7ff,Still_avaliable,C2-7287916l,0.48
2,2,Tiffany's World Pups Picnic Playset,7.00,toys-and-games,1945816904649,1997-03-29,b089ef6f-b628-4e37-811d-fffe0102ba64,Still_avaliable,S7-1175877v,0.59
3,3,Tiffany's World Wildlife Park Adventures,12.99,toys-and-games,1569790890899,2013-03-20,d55de422-8b98-47d6-9991-e4bc4c5c0cb0,Removed,D8-8421505n,0.54
4,4,Cosatto Cosy Dolls Pram,30.00,toys-and-games,7142740213920,2007-12-23,7945b657-cb02-4cc5-96cf-f65ed0a8f235,Still_avaliable,B6-2596063a,1.91


### Connect to Local Database and Upload Cleaned Data

In [20]:
# Step 6: Connect to the Local Database
if 'cleaned_df' in locals():
    local_db_connector = DatabaseConnector(config_path='local_db_creds.yaml')

    # Step 7: Upload the cleaned data to the local database
    local_db_connector.upload_to_db(cleaned_df, "dim_products")
    print("Cleaned data uploaded to the local database as 'dim_products'")

    # Upload the legacy_dim_products dataframe
    legend_dim_products = pd.read_csv('legacy_dim_products.csv')
    local_db_connector.upload_to_db(legend_dim_products, "legacy_dim_products")
    print("Legend data uploaded to the local database as 'legacy_dim_products'")
else:
    print("Cleaned data not available, cannot upload.")

Database engine initialized successfully.
Data uploaded to table dim_products successfully.
Cleaned data uploaded to the local database as 'dim_products'
Data uploaded to table legacy_dim_products successfully.
Legend data uploaded to the local database as 'legacy_dim_products'


## TASK 5


### Connect to AWS RDS Database

In [21]:
# Step 1: Connect to the AWS RDS Database
rds_db_connector = DatabaseConnector(config_path='aws_db_creds.yaml')

Database engine initialized successfully.


### Extract Data from RDS

In [22]:
# Step 2: Extract Data from RDS
data_extractor = DataExtractor(rds_db_connector)
orders_df = data_extractor.read_rds_table('orders_table')

# Check if the DataFrame was retrieved successfully
if orders_df is not None:
    print("Data before cleaning:")
    display(orders_df.head())  # Use display to render the DataFrame in Jupyter
    # Save the DataFrame as 'legacy_fact_orders'
    orders_df.to_csv('legacy_fact_orders.csv', index=False)
    print("DataFrame saved as 'legacy_fact_orders.csv'")
else:
    print("Failed to retrieve data from RDS.")

Data before cleaning:


,level_0,index,date_uuid,first_name,last_name,user_uuid,card_number,store_code,product_code,1,product_quantity
0,0,0,9476f17e-5d6a-4117-874d-9cdb38ca1fa6,None,None,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8,30060773296197,BL-8387506C,R7-3126933h,None,3
1,1,1,0423a395-a04d-4e4a-bd0f-d237cbd5a295,None,None,8fe96c3a-d62d-4eb5-b313-cf12d9126a49,349624180933183,WEB-1388012W,C2-7287916l,None,2
2,2,2,65187294-bb16-4519-adc0-787bbe423970,None,None,fc461df4-b919-48b2-909e-55c95a03fe6b,3529023891650490,CH-01D85C8D,S7-1175877v,None,2
3,3,3,579e21f7-13cb-436b-83ad-33687a4eb337,None,None,6104719f-ef14-4b09-bf04-fb0c4620acb0,213142929492281,CL-C183BE4B,D8-8421505n,None,2
4,4,4,00ab86c3-2039-4674-b9c1-adbcbbf525bd,None,None,9523a6d3-b2dd-4670-a51a-36aebc89f579,502067329974,SO-B5B9CB3B,B6-2596063a,None,2


DataFrame saved as 'legacy_fact_orders.csv'


### Clean the Orders Data

In [23]:
# Step 3: Clean the Data
if 'orders_df' in locals():
    data_cleaner = DataCleaning()
    cleaned_df = data_cleaner.clean_orders_data(orders_df)

    # Step 4: Display the cleaned data
    print("Data after cleaning:")
    display(cleaned_df.head())
else:
    print("Orders data not available, cannot proceed with cleaning.")

Data after cleaning:


,level_0,index,date_uuid,user_uuid,card_number,store_code,product_code,product_quantity
0,0,0,9476f17e-5d6a-4117-874d-9cdb38ca1fa6,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8,30060773296197,BL-8387506C,R7-3126933h,3
1,1,1,0423a395-a04d-4e4a-bd0f-d237cbd5a295,8fe96c3a-d62d-4eb5-b313-cf12d9126a49,349624180933183,WEB-1388012W,C2-7287916l,2
2,2,2,65187294-bb16-4519-adc0-787bbe423970,fc461df4-b919-48b2-909e-55c95a03fe6b,3529023891650490,CH-01D85C8D,S7-1175877v,2
3,3,3,579e21f7-13cb-436b-83ad-33687a4eb337,6104719f-ef14-4b09-bf04-fb0c4620acb0,213142929492281,CL-C183BE4B,D8-8421505n,2
4,4,4,00ab86c3-2039-4674-b9c1-adbcbbf525bd,9523a6d3-b2dd-4670-a51a-36aebc89f579,502067329974,SO-B5B9CB3B,B6-2596063a,2


### Connect to Local Database and Upload Cleaned Data

In [24]:
# Step 5: Connect to the Local Database
if 'cleaned_df' in locals():
    local_db_connector = DatabaseConnector(config_path='local_db_creds.yaml')

    # Step 6: Upload the cleaned data to the local database
    local_db_connector.upload_to_db(cleaned_df, "orders_table")
    print("Cleaned data uploaded to the local database as 'orders_table'")

    # Upload the legacy_orders_table dataframe
    legend_orders_table = pd.read_csv('legacy_fact_orders.csv')
    local_db_connector.upload_to_db(legend_orders_table, "legacy_orders_table")
    print("Legend data uploaded to the local database as 'legacy_orders_table'")
else:
    print("Cleaned data not available, cannot upload.")

Database engine initialized successfully.
Data uploaded to table orders_table successfully.
Cleaned data uploaded to the local database as 'orders_table'
Data uploaded to table legacy_orders_table successfully.
Legend data uploaded to the local database as 'legacy_orders_table'


## TASK 6


### Define JSON URL and Filename, Fetch and Save JSON Data

In [25]:
# Step 1: Fetch the JSON data from the S3 URL and save it to a file
json_url = "https://data-handling-public.s3.eu-west-1.amazonaws.com/date_details.json"
filename = "date_details.json"

# Create an instance of DataCleaning to handle JSON operations
data_cleaner = DataCleaning()

# Fetch and save the JSON data
raw_json_data = data_cleaner.fetch_and_save_json(json_url, filename)

# Check if the JSON data was successfully retrieved
if raw_json_data:
    print("Raw JSON data fetched and saved successfully.")
else:
    print("Failed to fetch JSON data from the given URL.")

Raw JSON data fetched and saved successfully.


### Clean the JSON Data

In [26]:
# Step 2: Clean the saved JSON data
if raw_json_data:
    cleaned_df = data_cleaner.clean_date_events_data(raw_json_data)

    # Display the cleaned data if available
    if cleaned_df is not None:
        print("Final cleaned data:")
        display(cleaned_df.head())  # Use display() for better visualization in Jupyter
    else:
        print("Failed to clean the JSON data.")
else:
    print("JSON data not available, cannot proceed with cleaning.")

Data after reformatting to DataFrame:
  timestamp month  year day time_period                             date_uuid
0  22:00:06     9  2012  19     Evening  3b7ca996-37f9-433f-b6d0-ce8391b615ad
1  22:44:06     2  1997  10     Evening  adc86836-6c35-49ca-bb0d-65b6507a00fa
2  10:05:37     4  1994  15     Morning  5ff791bf-d8e0-4f86-8ceb-c7b60bef9b31
3  17:29:27    11  2001   6      Midday  1b01fcef-5ab9-404c-b0d4-1e75a0bd19d8
4  22:40:33    12  2015  31     Evening  dfa907c1-f6c5-40f0-aa0d-40ed77ac5a44
Data after removing null rows:
  timestamp month  year day time_period                             date_uuid
0  22:00:06     9  2012  19     Evening  3b7ca996-37f9-433f-b6d0-ce8391b615ad
1  22:44:06     2  1997  10     Evening  adc86836-6c35-49ca-bb0d-65b6507a00fa
2  10:05:37     4  1994  15     Morning  5ff791bf-d8e0-4f86-8ceb-c7b60bef9b31
3  17:29:27    11  2001   6      Midday  1b01fcef-5ab9-404c-b0d4-1e75a0bd19d8
4  22:40:33    12  2015  31     Evening  dfa907c1-f6c5-40f0-aa0d-40ed77ac

,time_period,date_uuid,datetime
0,Evening,3b7ca996-37f9-433f-b6d0-ce8391b615ad,2012-09-19 22:00:06
1,Evening,adc86836-6c35-49ca-bb0d-65b6507a00fa,1997-02-10 22:44:06
2,Morning,5ff791bf-d8e0-4f86-8ceb-c7b60bef9b31,1994-04-15 10:05:37
3,Midday,1b01fcef-5ab9-404c-b0d4-1e75a0bd19d8,2001-11-06 17:29:27
4,Evening,dfa907c1-f6c5-40f0-aa0d-40ed77ac5a44,2015-12-31 22:40:33


### Connect to Local Database and Upload Cleaned Data

In [27]:
# Step 3: Connect to the Local Database and Upload the Cleaned Data
if 'cleaned_df' in locals():
    # Create an instance of DatabaseConnector
    local_db_connector = DatabaseConnector(config_path='local_db_creds.yaml')

    # Step 4: Upload the cleaned data to the local database as 'dim_date_times'
    local_db_connector.upload_to_db(cleaned_df, "dim_date_times")
    print("Cleaned data uploaded to the local database as 'dim_date_times'")
else:
    print("Cleaned data not available, cannot upload.")

Database engine initialized successfully.
Data uploaded to table dim_date_times successfully.
Cleaned data uploaded to the local database as 'dim_date_times'
